# Convert SQLite outputs to parquet files with cytotable

## Import libraries

In [1]:
import pathlib
import pandas as pd

# cytotable will merge objects from SQLite file into single cells and save as parquet file
from cytotable import convert, presets

import logging

# Set the logging level to a higher level to avoid outputting unnecessary errors from config file in convert function
logging.getLogger().setLevel(logging.ERROR)

## Set paths and variables

In [2]:
# preset configurations based on typical CellProfiler outputs
preset = "cellprofiler_sqlite_pycytominer"

# update preset to include site metadata and cell counts
joins = presets.config["cellprofiler_sqlite_pycytominer"]["CONFIG_JOINS"].replace(
    "Image_Metadata_Well,",
    "Image_Metadata_Well, Image_Metadata_Site, Image_Count_Cells,",
)

# type of file output from cytotable (currently only parquet)
dest_datatype = "parquet"

# set path to directory with SQLite files
sqlite_dir = pathlib.Path("../2.cellprofiler_processing/cp_output")

# directory for processed data
output_dir = pathlib.Path("data")
output_dir.mkdir(parents=True, exist_ok=True)

plate_names = []

for file_path in sqlite_dir.iterdir():
    plate_names.append(file_path.stem)

# print the plate names and how many plates there are (confirmation)
print(f"There are {len(plate_names)} plates in this dataset. Below are the names:")
for name in plate_names:
    print(name)

There are 4 plates in this dataset. Below are the names:
localhost230405150001
localhost220512140003_KK22-05-198
localhost231120090001
localhost220513100001_KK22-05-198_FactinAdjusted


## Convert SQLite to parquet files

In [3]:
for file_path in sqlite_dir.iterdir():
    output_path = pathlib.Path(f"{output_dir}/converted_profiles/{file_path.stem}_converted.parquet")
    print("Starting conversion with cytotable for plate:", file_path.stem)
    # merge single cells and output as parquet file
    convert(
        source_path=str(file_path),
        dest_path=str(output_path),
        dest_datatype=dest_datatype,
        preset=preset,
        joins=joins,
        chunk_size=5000
    )
    
print("All plates have been converted with cytotable!")

Starting conversion with cytotable for plate: localhost230405150001
Starting conversion with cytotable for plate: localhost220512140003_KK22-05-198
Starting conversion with cytotable for plate: localhost231120090001
Starting conversion with cytotable for plate: localhost220513100001_KK22-05-198_FactinAdjusted
All plates have been converted with cytotable!


# Load in converted profiles to update

In [4]:
# Directory with converted profiles
converted_dir = pathlib.Path(f"{output_dir}/converted_profiles")

for file_path in converted_dir.iterdir():
    # Load the DataFrame from the Parquet file
    df = pd.read_parquet(file_path)
    
    # Drop rows where "Metadata_ImageNumber" is NaN (artifact of cytotable)
    df = df.dropna(subset=["Metadata_ImageNumber"])
    
    # Columns to move to the front
    columns_to_move = ['Cells_Location_Center_X', 'Cells_Location_Center_Y', "Image_Count_Cells"]
    
    # Rearrange columns and add "Metadata" prefix in one line
    df = (df[columns_to_move + [col for col in df.columns if col not in columns_to_move]]
                  .rename(columns=lambda col: 'Metadata_' + col if col in columns_to_move else col))
    
    # Save the processed DataFrame as Parquet in the same path
    df.to_parquet(file_path, index=False)


## Check output to confirm process worked

To confirm the number of single cells is correct, please use any database browser software to see if the number of rows in the "Per_Cells" compartment matches the number of rows in the data frame.

In [5]:
converted_df = pd.read_parquet("./data/converted_profiles/localhost220512140003_KK22-05-198_converted.parquet")

print(converted_df.shape)
converted_df.head()

(42288, 2016)


,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Image_Metadata_Plate,Image_Metadata_Site,Image_Metadata_Well,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,...,Nuclei_Texture_Variance_Hoechst_3_02_256,Nuclei_Texture_Variance_Hoechst_3_03_256,Nuclei_Texture_Variance_Mitochondria_3_00_256,Nuclei_Texture_Variance_Mitochondria_3_01_256,Nuclei_Texture_Variance_Mitochondria_3_02_256,Nuclei_Texture_Variance_Mitochondria_3_03_256,Nuclei_Texture_Variance_PM_3_00_256,Nuclei_Texture_Variance_PM_3_01_256,Nuclei_Texture_Variance_PM_3_02_256,Nuclei_Texture_Variance_PM_3_03_256
0,238.472579,106.096854,34,1.0,localhost220512140003_KK22-05-198,f00,A01,1.0,1.0,4.0,...,5.049532,5.034654,0.668727,0.618992,0.579687,0.565164,0.403016,0.403598,0.406572,0.401812
1,178.623098,192.302264,34,1.0,localhost220512140003_KK22-05-198,f00,A01,4.0,4.0,7.0,...,3.262048,3.114875,0.477114,0.430199,0.428481,0.435792,0.696155,0.667151,0.702374,0.721301
2,363.225655,247.140907,34,1.0,localhost220512140003_KK22-05-198,f00,A01,7.0,7.0,10.0,...,7.150995,7.098986,7.351778,8.275158,8.146412,6.910650,5.864586,6.120011,5.979275,5.644951
3,554.333503,255.714826,34,1.0,localhost220512140003_KK22-05-198,f00,A01,8.0,8.0,11.0,...,4.184469,4.148621,0.482569,0.473050,0.473857,0.484668,0.378843,0.372276,0.381904,0.376825
4,464.374900,278.254591,34,1.0,localhost220512140003_KK22-05-198,f00,A01,9.0,9.0,12.0,...,4.345724,4.479495,0.433022,0.402844,0.398733,0.398392,0.919997,0.771727,0.772588,0.794130
